# Roadway Network Search
The purpose of this notebook is to understand and visualize how Network Wrangler conducts searches. 
It can also be used as a template to troubleshoot or check specific searches.

In [4]:
import os
import warnings

import folium
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
from shapely.geometry import Point

from network_wrangler import RoadwayNetwork
from network_wrangler import ProjectCard

%config IPCompleter.greedy=True
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
#if you need to reset variables in the notebook
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [7]:
import logging
import sys
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout

# set to logging.INFO for less detail or logging.DEBUG for more
logger.setLevel(logging.DEBUG)

# 1 - Load Network

 - Read the network files into the `RoadwayNetwork` class

In [8]:
STPAUL_DIR = os.path.join(os.path.dirname(os.getcwd()),'examples','stpaul')
STPAUL_SHAPE_FILE = os.path.join(STPAUL_DIR,"shape.geojson")
STPAUL_LINK_FILE = os.path.join(STPAUL_DIR,"link.json")
STPAUL_NODE_FILE = os.path.join(STPAUL_DIR,"node.geojson")

In [9]:
net = RoadwayNetwork.read(
    link_file= STPAUL_LINK_FILE, 
    node_file=STPAUL_NODE_FILE, 
    shape_file=STPAUL_SHAPE_FILE, 
    fast=True,
    shape_foreign_key ='shape_id'
)

2020-11-13 16:03:23, INFO: Reading RoadwayNetwork
2020-11-13 16:03:23, INFO: Reading RoadwayNetwork
2020-11-13 16:03:23, DEBUG: Reading RoadwayNetwork from following files:
   -/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/link.json
   -/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/node.geojson
   -/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/shape.geojson.
2020-11-13 16:03:29, INFO: Read 66253 links from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/link.json
2020-11-13 16:03:29, INFO: Read 66253 links from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/link.json
2020-11-13 16:03:29, INFO: Read 17159 nodes from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/examples/stpaul/node.geojson
2020-11-13 16:03:29, INFO: Read 17159 nodes from /Users/elizabe

# 2 - Define Selections
  - can be represented by dictionaries
  - `link.name` is required unless a `link` unique ID is used (set at `RoadwayNetwork.UNIQUE_MODEL_LINK_IDENTIFIERS`)
  - A and B nodes should be set with a unique identifier
  - If you want to qualify your selection farther based on attributes, you can add that as well

In [10]:
easy_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        ],
     'A':{'osm_node_id': '187899923'}, # start searching for segments at A
     'B':{'osm_node_id': '187865924'}, # end at B
}

longer_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        ],
     'A':{'osm_node_id': '187899923'}, # start searching for segments at A
     'B':{'osm_node_id': '187942339'}, # end at B
}

multi_criteria_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        {'lanes': [1,2]} # from the initial selection, only select streets with 1 or 2 lanes
         ],
     'A':{'osm_node_id': '187899923'}, # start searching for segments at A
     'B':{'osm_node_id': '187942339'}, # end at B
}

### Selection Query

This is what the selection looks like when translated into a query.

In [11]:
S = multi_criteria_selection
ProjectCard.build_link_selection_query(S, unique_model_link_identifiers=RoadwayNetwork.UNIQUE_MODEL_LINK_IDENTIFIERS, ignore=['name'])

AttributeError: type object 'RoadwayNetwork' has no attribute 'UNIQUE_MODEL_LINK_IDENTIFIERS'

# 3 - Run Selections

  - Selections are run by feeding the selection dictionary into `select_roadway_features()` and are stored in a key which can be created by running `build_selection_key()`

In [12]:
# default is 10, but we may want to set it higher here for debugging purposes
# in reality, we probably want to set it lower
RoadwayNetwork.MAX_SEARCH_BREADTH = 15

net.select_roadway_features(easy_selection)
net.select_roadway_features(longer_selection)
net.select_roadway_features(multi_criteria_selection)

easy_key = net.build_selection_key(easy_selection)
longer_key = net.build_selection_key(longer_selection)
multi_key = net.build_selection_key(multi_criteria_selection)                                              

2020-11-13 16:11:07, DEBUG: validating selection
2020-11-13 16:11:07, DEBUG: Selection Key: ('((name.str.contains("6th") or name.str.contains("Sixth") or name.str.contains("sixth")) and (drive_access==1))', 62146, 45691)
2020-11-13 16:11:07, DEBUG: Building selection query
2020-11-13 16:11:07, DEBUG: Selecting features:
((name.str.contains("6th") or name.str.contains("Sixth") or name.str.contains("sixth")) and (drive_access==1))
2020-11-13 16:11:07, DEBUG: Completed query
2020-11-13 16:11:07, DEBUG: Not a unique ID selection, conduct search.
2020-11-13 16:11:07, DEBUG: Initial set of nodes: [39425, 68609, 41991, 7688, 77072, 77073, 77075, 77077, 4758, 77078, 4759, 4760, 69793, 84899, 100782, 100784, 11188, 70841, 62146, 4806, 4807, 46666, 74961, 28505, 28513, 28515, 28516, 37609, 72303, 45683, 45685, 74997, 45687, 45688, 45691]
2020-11-13 16:11:07, DEBUG: calculating shortest path from graph
2020-11-13 16:11:07, DEBUG: Calculating shortest path from 62146 to 45691 using i as weight wit

2020-11-13 16:11:10, DEBUG: Adding breadth, with shortest path iteration. i: 1 Max i: 5
2020-11-13 16:11:10, DEBUG: -Adding Breadth-
2020-11-13 16:11:10, DEBUG: Candidate Nodes: 325
2020-11-13 16:11:10, DEBUG: Adding 797 links.
2020-11-13 16:11:10, DEBUG: Calculating shortest path from 62146 to 77073 using i as weight with a factor of 100
2020-11-13 16:11:10, DEBUG: Creating network graph
2020-11-13 16:11:10, DEBUG: starting ox_graph()
2020-11-13 16:11:10, DEBUG: GRAPH NODES: Index(['model_node_id', 'osm_node_id', 'shstReferenceId', 'drive_node',
       'walk_node', 'bike_node', 'bus_only', 'rail_only', 'geometry', 'X',
       'Y'],
      dtype='object')
2020-11-13 16:11:10, DEBUG: starting ox.gdfs_to_graph()
2020-11-13 16:11:10, DEBUG: finished ox.gdfs_to_graph()
2020-11-13 16:11:10, DEBUG: No SP from 62146 to 77073 Found.
2020-11-13 16:11:10, DEBUG: Adding breadth, with shortest path iteration. i: 2 Max i: 5
2020-11-13 16:11:10, DEBUG: -Adding Breadth-
2020-11-13 16:11:10, DEBUG: Can

## Search Graph
When Network Wrangler conducts a search, it tries to navigate from A to B. If it cannot do so initially, it will expand its search graph several time until it achieves a navigable route – or reaches the maximum number of expansions which defaults to 10 but can be set higher by running `RoadwayNetwork.MAX_SEARCH_BREADTH = 15`. In production, you will want to set the search breadth low so that you don't get strange routes. 

The graphs for the searches are stored in the `graph` key of the `selections` attribute of the RoadwayNetwork class  in Network X format so they can be used in the future. 

In [13]:
G = net.selections[easy_key]['graph']
#G.edges
#nx.get_edge_attributes(G,'weight')

# 4 - Visualizing the Selection

Selection visualization has been thrown into the class defined below `selection_map`.  In the future, this should be generalized and moved into Network Wrangler.

In [17]:
class selection_map:
    
    def __init__(self, net, selection={}):
        self.net = net
        self.selection = selection
        self.query = ProjectCard.build_link_selection_query(selection,unique_link_ids=net.unique_link_ids)
        
        self.selection_key = net.build_selection_key(self.selection)
        
        self.G     = self.net.selections[self.selection_key]['graph']
        self.links = self.net.selections[self.selection_key]['candidate_links']
        self.A_name = self.selection_key[1]
        self.B_name = self.selection_key[2]
        
        self.A     = self.G.nodes[self.A_name]
        self.B     = self.G.nodes[self.B_name]
        
        self.sel_found = self.net.selections[self.selection_key]['selection_found']
        if self.sel_found:
            self.selected_links = self.net.selections[self.selection_key]['selected_links']
            self.sp_route = self.net.selections[self.selection_key]['route']        
    
    def folium_node(self, node, node_name, color='white', icon = ''):
        node_marker = folium.Marker(location=[node['Y'],node['X']], 
                                    icon=folium.Icon(icon = icon, color=color), 
                                    tooltip=node_name,
                                   )
        return node_marker
    
    def folium_route(self):
        '''
        shows the sp route along which the selection is being created
        '''
        m = ox.plot_route_folium(self.G, self.sp_route, route_map=self.folium_graph(),
                          route_color='white', route_width=15, route_opacity=.5)
        return m
    
    def folium_selection(self):
        '''
        Shows which links are selected
        '''
        m = self.folium_graph()
        
        for _, row in self.net.selections[self.selection_key]['selected_links'].iterrows():
            pl = ox.folium._make_folium_polyline(edge=row, edge_color="green", edge_width=7,
                                      edge_opacity=0.4)
            pl.add_to(m)
            
        
        return m
    
    def folium_graph(self):
        '''
        Shows the entirety of the search subgraph
        '''
        m = ox.plot_graph_folium(self.G, 
                                 edge_color='blue',
                                 edge_width=5, 
                                 edge_opacity = 0.1,
                                 tiles='cartodbdark_matter')
        self.folium_node(self.A, self.A_name, color="green", icon = 'play').add_to(m)
        self.folium_node(self.B, self.B_name, color="pink", icon = 'star').add_to(m)
        
        return m

    
    def candidate_graph(self):
        '''
        To review in case it doesn't find the SP
        ##todo doesn't work.
        '''
        m = self.folium_graph()
        
        links = ox.utils_graph.graph_to_gdfs(self.G, nodes=False, fill_edge_geometry=True)
        colors = ox.plot.get_colors(10)
        colors_d = {c:value for c, value in enumerate(colors, 0)}
        links['color'] = links['i'].map(colors_d)
        #print(links[['i','color']])
        
        for _, row in links.iterrows():
            pl = ox.folium._make_folium_polyline(edge=row, edge_color=row['color'], edge_width=5,
                                      edge_opacity=1.0)
            pl.add_to(m)
        self.folium_node(self.A, self.A_name, color="green").add_to(m)
        self.folium_node(self.B, self.B_name, color="pink").add_to(m)
        
        return m
    
sel = selection_map(net, selection=easy_selection)

## Display Selection

Showing final selected links in green and candidate links in blue.

In [18]:
sel.folium_selection()

#### Selection Candidates

From selection attributes before selecting from A to B

In [19]:
sel.candidate_graph()

Candidate links are stored as well.

In [20]:
sel.links[['model_link_id','A','B','name']]

,model_link_id,A,B,name
40,2846,4758,4806,East 6th Street
52,2918,4806,72303,East 6th Street
53,2919,4806,4807,East 6th Street
148,8426,7688,45687,West 6th Street
201,15110,11188,84899,"North Broadway Street,East 6th Street"
434,53998,28505,77075,East 6th Street
435,53999,28505,28515,E 6th Street
442,54019,28513,77078,East 6th Street
443,54020,28513,28516,E 6th Street
445,54026,28515,28513,E 6th Street


#### Route Map

Showing selected links from A to B.

In [21]:
sel.folium_route()